### Uploading Files to S3

1. Ensure your boto3 session is established and working
2. Required sales_data csv file is in the source folder

In [2]:
#import the libraries, if not installed then run pip3 install -r requirements.txt to install them
import boto3
import awswrangler as wr
import pandas as pd
import configparser
import warnings
warnings.filterwarnings('ignore')
import psycopg2
from io import StringIO
import configparser


In [3]:
#reading the credentials securely.
credentials = configparser.ConfigParser()

In [4]:
#use read_file method
credentials.read_file(open('credentials.config'))

In [5]:
#Reading in the credentials into Python variables. No one can see them
aws_key = credentials["AWS"]["KEY"]
aws_secret = credentials["AWS"]["SECRET"]
region = credentials["AWS"]["REGION"]

In [99]:
#Creating the Session
your_session = boto3.Session(aws_access_key_id=aws_key,
                            aws_secret_access_key=aws_secret,
                            region_name=region)
glue = boto3.client('glue', region)

In [107]:
#create variable with destination bucket path
destination = 's3://shellsalespipelinedata'

In [108]:
### create variable with source data path

In [109]:
local_file='source_folder'

In [110]:
#Confirming data in source_folder/sales_data.csv exists
!ls source_folder/

clean_datasales_data.csv  Normalized  sales_data.csv


In [111]:
#list objects destination folder to confirm it's empty
wr.s3.list_objects(path=destination,boto3_session=your_session)

['s3://shellsalespipelinedata/cleaned/clean_data.csv',
 's3://shellsalespipelinedata/sales_data.csv']

### Using wr.s3.upload() method

In [112]:
wr.s3.upload(local_file +'/sales_data.csv',
             path=destination + '/sales_data.csv',
            boto3_session=your_session)

In [113]:
### Verifying the file has uploaded successfully using list_objects()

In [114]:
wr.s3.list_objects(path=destination,boto3_session=your_session)

['s3://shellsalespipelinedata/cleaned/clean_data.csv',
 's3://shellsalespipelinedata/sales_data.csv']

In [115]:
### Loading csv file into a dataframe

In [116]:
sales_csv = pd.read_csv("source_folder/sales_data.csv")

In [117]:
### Remove duplicate rows

In [118]:
sales_df_cleaned = sales_csv.drop_duplicates()

In [119]:
### Remove rows with any null values

In [120]:
df_cleaned = sales_df_cleaned.dropna()

In [121]:
# print(df_cleaned.isnull().sum())
duplicates = df_cleaned.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [122]:
### Convert the DataFrame back to CSV format

In [123]:
cleaned_local_file_path = local_file +'/clean_datasales_data.csv'
csv_cleaned = df_cleaned.to_csv(cleaned_local_file_path,index=False)

In [124]:
### Upload the cleaned CSV back to S3 cleaned folder

In [125]:
wr.s3.upload(local_file +'/clean_datasales_data.csv',
             path=destination + '/cleaned/clean_data.csv',
            boto3_session=your_session)

## Normalization been done on the cleaned_data files uploaded in folder named normalized in s3

In [130]:
# Load the dataset (for example from S3)
s3_bucket = 'your-bucket'
s3_key = destination + '/clean_datasales_data'

data = wr.s3.read_csv(path=s3_key, boto3_session=your_session)


# Display the first few rows of the data
data.head()


,Distributor Name,DC Number,DC Name,Sales Month,Sales Day,Sales Year,Distributor SKU,Shell SKU Number,Distributor SKU Description,DFOA Quantity,Non-DFOA Quantity,Unit of Measure
0,X Petroleum,1001,Warehouse A,8,31,2024,ABC123-GL,1,Product A Description,1000,0,GL
1,X Petroleum,1001,Warehouse A,8,31,2024,DEF456-GL,2,Product B Description,188,2204,GL
2,X Petroleum,1002,Warehouse B,8,31,2024,DEF456-GL,2,Product B Description,150,1709,GL
3,X Petroleum,1001,Warehouse A,8,31,2024,GHI789-D55GL,3,Product C Description,4,1,D55GL
4,X Petroleum,1002,Warehouse B,8,31,2024,JKL012-D55GL,4,Product D Description,3,0,D55GL


In [142]:
# Create Distributor Table
distributor_df = pd.DataFrame({
    'DistributorID': [1],
    'Distributor Name': ['X Petroleum']
})

# Create DC Table
dc_df = data[['DC Number', 'DC Name']].drop_duplicates().reset_index(drop=True)
dc_df['DCID'] = dc_df.index + 1
dc_df['DistributorID'] = 1

# Create Sales Date Table
date_df = data[['Sales Month', 'Sales Day', 'Sales Year']].drop_duplicates().reset_index(drop=True)
date_df['DateID'] = date_df.index + 1

# Create SKU Table
sku_df = data[['Distributor SKU', 'Shell SKU Number', 'Distributor SKU Description']].drop_duplicates().reset_index(drop=True)
sku_df['SKU_ID'] = sku_df.index + 1

# Create Sales Table
sales_df = data[['DC Number', 'Distributor SKU', 'DFOA Quantity', 'Non-DFOA Quantity', 'Unit of Measure']]
sales_df['SaleID'] = sales_df.index + 1

# Merge to create foreign key relationships
sales_df = sales_df.merge(dc_df[['DC Number', 'DCID']], on='DC Number', how='left')
sales_df = sales_df.merge(sku_df[['Distributor SKU', 'SKU_ID']], on='Distributor SKU', how='left')
sales_df['DateID'] = 1  # Assuming all sales happen on the same date

# Final DataFrames
print("Distributor Table:")
print(distributor_df)
print("\nDC Table:")
print(dc_df)
print("\nSales Date Table:")
print(date_df)
print("\nSKU Table:")
print(sku_df)
print("\nSales Table:")
print(sales_df)
# wr.config.init_session(your_session)


Distributor Table:
   DistributorID Distributor Name
0              1      X Petroleum

DC Table:
   DC Number      DC Name  DCID  DistributorID
0       1001  Warehouse A     1              1
1       1002  Warehouse B     2              1

Sales Date Table:
   Sales Month  Sales Day  Sales Year  DateID
0            8         31        2024       1

SKU Table:
   Distributor SKU  Shell SKU Number Distributor SKU Description  SKU_ID
0        ABC123-GL                 1       Product A Description       1
1        DEF456-GL                 2       Product B Description       2
2     GHI789-D55GL                 3       Product C Description       3
3     JKL012-D55GL                 4       Product D Description       4
4     MNO345-D55GL                 5       Product E Description       5
5        PQR678-GL                 6       Product F Description       6
6     STU901-D209L                 7       Product G Description       7
7     VWX234-D55GL                 8       Product H D

In [136]:

# Temporary local CSV files
local_distributor_path = '/tmp/distributor_table.csv'
local_dc_path = '/tmp/dc_table.csv'
local_date_path = '/tmp/sales_date_table.csv'
local_sku_path = '/tmp/sku_table.csv'
local_sales_path = '/tmp/sales_table.csv'

# Save DataFrames to local CSV files
distributor_df.to_csv(local_distributor_path, index=False)
dc_df.to_csv(local_dc_path, index=False)
date_df.to_csv(local_date_path, index=False)
sku_df.to_csv(local_sku_path, index=False)
sales_df.to_csv(local_sales_path, index=False)

# Upload local CSV files to S3
wr.s3.upload(local_distributor_path, path=destination + '/uploadedvianotebooks/distributor_table.csv', boto3_session=your_session)
wr.s3.upload(local_distributor_path, path=destination + '/uploadedvianotebooks/dc_table.csv', boto3_session=your_session)
wr.s3.upload(local_distributor_path, path=destination + '/uploadedvianotebooks/sales_date_table.csv', boto3_session=your_session)
wr.s3.upload(local_distributor_path, path=destination + '/uploadedvianotebooks/sku_table.csv', boto3_session=your_session)
wr.s3.upload(local_distributor_path, path=destination + '/uploadedvianotebooks/sales_table.csv', boto3_session=your_session)


# # You may clean up local files if needed
# os.remove(local_distributor_path)
# os.remove(local_dc_path)
# os.remove(local_date_path)
# os.remove(local_sku_path)
# os.remove(local_sales_path)

print("DataFrames uploaded successfully to S3!")

DataFrames uploaded successfully to S3!


### Load data from s3 to RDS Postgresql

In [163]:
### schema for the rds tables

In [ ]:
#  1. Create Distributor Table
# CREATE TABLE Distributor (
#     DistributorID SERIAL PRIMARY KEY,
#     DistributorName VARCHAR(255) NOT NULL
# );

#  2. Create DC (Distribution_Center) Table
# CREATE TABLE DC (
#     DCID SERIAL PRIMARY KEY,
#     DCNumber VARCHAR(10) NOT NULL,
#     DCName VARCHAR(255) NOT NULL,
#     DistributorID INT REFERENCES Distributor(DistributorID)
# );

#  3. Create Sales_Date Table
# CREATE TABLE SalesDate (
#     DateID SERIAL PRIMARY KEY,
#     SalesMonth INT NOT NULL,
#     SalesDay INT NOT NULL,
#     SalesYear INT NOT NULL
# );

#  4. Create SKU Table
# CREATE TABLE SKU (
#     SKU_ID SERIAL PRIMARY KEY,
#     DistributorSKU VARCHAR(50) NOT NULL,
#     ShellSKUNumber INT NOT NULL,
#     DistributorSKUDescription VARCHAR(255) NOT NULL
# );

#  5. Create Sales Table
# CREATE TABLE Sales (
#     SaleID SERIAL PRIMARY KEY,
#     DCID INT REFERENCES DC(DCID),
#     DateID INT REFERENCES SalesDate(DateID),
#     SKU_ID INT REFERENCES SKU(SKU_ID),
#     DFOAQuantity INT NOT NULL,
#     NonDFOAQuantity INT NOT NULL,
#     UnitOfMeasure VARCHAR(50) NOT NULL
# );


In [6]:
import awswrangler as wr
import pandas as pd
from io import StringIO
import psycopg2
from psycopg2 import pool

# Example of setting up a connection pool
connection_pool = pool.SimpleConnectionPool(1, 10, 
    user=rds_username,
    password=rds_password,
    host=rds_host,
    port=rds_port,
    database=rds_dbname
)

def load_csv_to_postgres(table_name, boto3_session):
    # Load CSV data from S3 using awswrangler
    s3_key = f'{s3_prefix}{table_name}.csv'  # Construct the S3 key for the CSV file
    
    # Read the CSV into a DataFrame
    df = wr.s3.read_csv(path=f's3://{s3_bucket}/{s3_key}', boto3_session=boto3_session)

    # Get a connection from the pool
    connection = connection_pool.getconn()
    try:
        cursor = connection.cursor()
        
        # Create a StringIO object to hold CSV data for bulk insertion
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False, header=False)  # Write DataFrame to StringIO as CSV without header
        csv_buffer.seek(0)  # Move the cursor to the beginning of the StringIO buffer

        # Use the COPY command for efficient bulk insertion into PostgreSQL
        cursor.copy_from(csv_buffer, table_name, sep=',')  # Insert data from the buffer into the specified table
        
        connection.commit()  # Commit the transaction after successful insertion

    except Exception as e:
        connection.rollback()  # Rollback the transaction in case of error
        print(f"Error loading data into {table_name}: {e}")  # Print the error message

    finally:
        cursor.close()  # Ensure cursor is closed
        connection_pool.putconn(connection)  # Return the connection to the pool

# Load each table from the list into PostgreSQL
tables = ['DC', 'sales_table', 'Distributor', 'SalesDate','sku', 'sales']
for table in tables:
    load_csv_to_postgres(table, your_session)  # Call the function for each table

# No need to close cursor or connection here, as it's handled in the function

print("Data loaded successfully into PostgreSQL.")  # Confirmation message


### Test Query on RDS Postgresql

In [170]:
import configparser
import psycopg2  # Use pymysql for MySQL
import pandas as pd

# Read the configuration file
config = configparser.ConfigParser()
config.read('credent.config')

# Database using env variables connection parameters
db_params = {
    'host': config['database']['host'],
    'database': config['database']['dbname'],
    'user': config['database']['username'],
    'password': config['database']['password'],
    'port': config['database'].getint('port')  # Convert to int
}

# Connect to the database
try:
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    print("Connected to the database")

    # Example query
    query = "SELECT * FROM Distributor;"
    cursor.execute(query)
    
    # Fetch the data into a pandas DataFrame
    data = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(data, columns=column_names)

    # Display the DataFrame
    print(df)

except Exception as e:
    print(f"Error: {e}")

finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()
        print("Connection closed")


Connected to the database
   distributorid distributorname
0              1     X Petroleum
Connection closed


In [177]:
### Create indexes and Materialized views for high performance queries before executing query tasks by Odete

In [183]:
import psycopg2
import configparser

# Read database credentials from config file
config = configparser.ConfigParser()
config.read('credent.config')

# Extract credentials
db_host = config['database']['host']
db_name = config['database']['dbname']
db_user = config['database']['username']
db_password = config['database']['password']

# Initialize connection and cursor
conn = None
cursor = None

try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )

    # Create a cursor object
    cursor = conn.cursor()

    # SQL commands
    sql_commands = """
    -- Create Indexes
    CREATE INDEX IF NOT EXISTS idx_dc_number ON DC(DCNumber);
    CREATE INDEX IF NOT EXISTS idx_distributor_name ON Distributor(DistributorName);
    CREATE INDEX IF NOT EXISTS idx_sales_date ON SalesDate(SalesMonth, SalesDay, SalesYear);
    CREATE INDEX IF NOT EXISTS idx_sku ON SKU(DistributorSKU);
    CREATE INDEX IF NOT EXISTS idx_dc_id ON Sales(DCID);
    CREATE INDEX IF NOT EXISTS idx_date_id ON Sales(DateID);
    CREATE INDEX IF NOT EXISTS idx_sku_id ON Sales(SKU_ID);

    -- Create Materialized View
    CREATE MATERIALIZED VIEW IF NOT EXISTS mv_sales_summary AS
    SELECT 
        d.DistributorName,
        dc.DCName,
        sd.SalesMonth,
        sd.SalesYear,
        s.SKU_ID,
        s.DFOAQuantity,
        s.NonDFOAQuantity,
        s.UnitOfMeasure
    FROM 
        Sales s
    JOIN 
        DC dc ON s.DCID = dc.DCID
    JOIN 
        Distributor d ON dc.DistributorID = d.DistributorID
    JOIN 
        SalesDate sd ON s.DateID = sd.DateID;
    """

    # Execute the SQL commands
    cursor.execute(sql_commands)

    # Commit the changes
    conn.commit()

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


### Executing client requirements


In [7]:
### 1. Calculate total sales revenue per product.We are missing vital data i.e unit price(priceperunit)
## TODO

In [184]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # SQL query to calculate total sales revenue per product
    revenue_query = """
    SELECT 
        sku.DistributorSKU,
        sku.DistributorSKUDescription,
        SUM(s.DFOAQuantity) AS Total_DFOA_Quantity,
        SUM(s.NonDFOAQuantity) AS Total_Non_DFOA_Quantity,
        (SUM(s.DFOAQuantity) + SUM(s.NonDFOAQuantity)) * COALESCE(sku.PricePerUnit, 0) AS Total_Revenue
    FROM 
        Sales s
    JOIN 
        SKU sku ON s.SKU_ID = sku.SKU_ID
    GROUP BY 
        sku.DistributorSKU, sku.DistributorSKUDescription
    ORDER BY 
        Total_Revenue DESC;
    """

    # Execute the query
    cursor.execute(revenue_query)
    
    # Fetch results
    results = cursor.fetchall()

    # Print results
    for row in results:
        print(f"SKU: {row[0]}, Description: {row[1]}, Total DFOA Quantity: {row[2]}, Total Non-DFOA Quantity: {row[3]}, Total Revenue: ${row[4]:.2f}")

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Database error occurred: column sku.priceperunit does not exist
LINE 7: ...FOAQuantity) + SUM(s.NonDFOAQuantity)) * COALESCE(sku.PriceP...
                                                             ^



In [194]:
### 2.Determine the total quantity sold per customer. Using Materialized View

In [192]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # SQL command to create the materialized view
    create_view_query = """
    CREATE MATERIALIZED VIEW IF NOT EXISTS mv_total_quantity_per_customer AS
    SELECT 
        d.DistributorName AS CustomerName,
        SUM(s.DFOAQuantity + s.NonDFOAQuantity) AS TotalQuantitySold
    FROM 
        Sales s
    JOIN 
        DC dc ON s.DCID = dc.DCID
    JOIN 
        Distributor d ON dc.DistributorID = d.DistributorID
    GROUP BY 
        d.DistributorName;
    """

    # Execute the command to create the materialized view
    cursor.execute(create_view_query)
    
    # Commit the changes
    conn.commit()

    # SQL command to refresh the materialized view
    refresh_view_query = "REFRESH MATERIALIZED VIEW mv_total_quantity_per_customer;"

    # Execute the command to refresh the materialized view
    cursor.execute(refresh_view_query)
    
    # Commit the changes
    conn.commit()

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


In [193]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # SQL query to fetch total quantity sold per customer
    fetch_query = "SELECT * FROM mv_total_quantity_per_customer;"

    # Execute the query
    cursor.execute(fetch_query)
    
    # Fetch results
    results = cursor.fetchall()

    # Print results
    for row in results:
        print(f"Customer: {row[0]}, Total Quantity Sold: {row[1]}")

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Customer: X Petroleum, Total Quantity Sold: 5283


In [8]:
### 3. Identify the top 5 products by revenue. Missing vital field priceperunit
##TODO

In [189]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # Step 1: Create the materialized view
    create_mv_query = """
    CREATE MATERIALIZED VIEW IF NOT EXISTS mv_product_revenue AS
    SELECT 
        sku.DistributorSKU,
        sku.DistributorSKUDescription,
        SUM(s.DFOAQuantity * COALESCE(sku.PricePerUnit, 0)) AS Total_DFOA_Revenue,
        SUM(s.NonDFOAQuantity * COALESCE(sku.PricePerUnit, 0)) AS Total_Non_DFOA_Revenue,
        SUM((s.DFOAQuantity + s.NonDFOAQuantity) * COALESCE(sku.PricePerUnit, 0)) AS Total_Revenue
    FROM 
        Sales s
    JOIN 
        SKU sku ON s.SKU_ID = sku.SKU_ID
    GROUP BY 
        sku.DistributorSKU, sku.DistributorSKUDescription;
    """
    
    # Execute the creation of the materialized view
    cursor.execute(create_mv_query)
    conn.commit()

    # Step 2: Query the materialized view to get the top 5 products by revenue
    top_products_query = """
    SELECT 
        DistributorSKU,
        DistributorSKUDescription,
        Total_DFOA_Revenue,
        Total_Non_DFOA_Revenue,
        Total_Revenue
    FROM 
        mv_product_revenue
    ORDER BY 
        Total_Revenue DESC
    LIMIT 5;
    """

    # Execute the query
    cursor.execute(top_products_query)
    
    # Fetch results
    results = cursor.fetchall()

    # Print results
    for row in results:
        print(f"SKU: {row[0]}, Description: {row[1]}, Total DFOA Revenue: ${row[2]:.2f}, Total Non-DFOA Revenue: ${row[3]:.2f}, Total Revenue: ${row[4]:.2f}")

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Database error occurred: column sku.priceperunit does not exist
LINE 6:         SUM(s.DFOAQuantity * COALESCE(sku.PricePerUnit, 0)) ...
                                              ^



In [195]:
### Generate a monthly sales report showing total revenue and quantity sold.

In [196]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # SQL command to create the monthly sales report view
    create_report_view_query = """
    CREATE MATERIALIZED VIEW IF NOT EXISTS mv_monthly_sales_report AS
    SELECT 
        sd.SalesYear,
        sd.SalesMonth,
        SUM(s.DFOAQuantity + s.NonDFOAQuantity) AS TotalQuantitySold,
        SUM((s.DFOAQuantity + s.NonDFOAQuantity) * COALESCE(sku.PricePerUnit, 0)) AS TotalRevenue
    FROM 
        Sales s
    JOIN 
        SKU sku ON s.SKU_ID = sku.SKU_ID
    JOIN 
        SalesDate sd ON s.DateID = sd.DateID
    GROUP BY 
        sd.SalesYear, sd.SalesMonth
    ORDER BY 
        sd.SalesYear, sd.SalesMonth;
    """

    # Execute the command to create the monthly sales report view
    cursor.execute(create_report_view_query)
    
    # Commit the changes
    conn.commit()

    # SQL command to refresh the materialized view
    refresh_view_query = "REFRESH MATERIALIZED VIEW mv_monthly_sales_report;"

    # Execute the command to refresh the materialized view
    cursor.execute(refresh_view_query)
    
    # Commit the changes
    conn.commit()

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Database error occurred: column sku.priceperunit does not exist
LINE 7: ...M((s.DFOAQuantity + s.NonDFOAQuantity) * COALESCE(sku.PriceP...
                                                             ^



In [197]:
try:
    # Establish the connection
    conn = psycopg2.connect(
        host=db_host,
        database=db_name,
        user=db_user,
        password=db_password
    )
    
    # Create a cursor object
    cursor = conn.cursor()
    
    # SQL query to fetch the monthly sales report
    fetch_report_query = "SELECT * FROM mv_monthly_sales_report;"

    # Execute the query
    cursor.execute(fetch_report_query)
    
    # Fetch results
    results = cursor.fetchall()

    # Print results
    for row in results:
        year, month, total_quantity, total_revenue = row
        print(f"Year: {year}, Month: {month}, Total Quantity Sold: {total_quantity}, Total Revenue: ${total_revenue:.2f}")

except psycopg2.DatabaseError as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the cursor and connection if they were opened
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Database error occurred: relation "mv_monthly_sales_report" does not exist
LINE 1: SELECT * FROM mv_monthly_sales_report;
                      ^

